<a href="https://colab.research.google.com/github/balasubramaniyan-p/pdsnd_github/blob/master/Index_RSI__R_MACD_Bolinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mftool==2.8

In [2]:

# import module
from mftool import Mftool
obj = Mftool()
scheme_code_dict=obj.get_scheme_codes()

In [3]:
def calculate_rsi(data, window,close="Close",date="Date",format="%d-%m-%Y"):
    # Convert 'Date' column to datetime
    data['Date'] = pd.to_datetime(data[date],format=format)
    data=data.sort_values(by='Date')
    data[close] = data[close].astype(float)
    data = data.reset_index(drop=True)
    delta = data[close].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [5]:
# Download historical data for NESI
import pandas as pd
tickers = ["120620"]

for ticker in tickers:
  try:
        data_pre = obj.get_scheme_historical_nav(code=ticker)
        data=pd.DataFrame(data_pre['data'])
        data['RSI'] = calculate_rsi(data, 14,date="date",close="nav",format="%d-%m-%Y")
        # Get the latest RSI value
        latest_rsi = data['RSI'].iloc[-1]
        print(f"\nThe latest RSI(14) value for {data_pre['scheme_name']} is {latest_rsi}.")
  except Exception as e:
    print(f"Error occurred while fetching data for {ticker}: {e}")


The latest RSI(14) value for ICICI Prudential Nifty 50 Index Fund - Direct Plan Cumulative Option is 73.33445235335154.
